In [ ]:
CREATE TABLE DbSecurity.UserAuthorization (
    UserAuthorizationKey INT NOT NULL PRIMARY KEY,
    ClassTime NCHAR(5) NULL CHECK (ClassTime IN ('10:45')),
    IndividualProject NVARCHAR(60) NULL DEFAULT 'PROJECT 2 RECREATE THE BICLASS DATABASE STAR SCHEMA',
    GroupMemberLastName NVARCHAR(35) NOT NULL,
    GroupMemberFirstName NVARCHAR(25) NOT NULL,
    GroupName NVARCHAR(20) NOT NULL,
    DateAdded DATETIME2 DEFAULT SYSDATETIME()
);

INSERT INTO DbSecurity.UserAuthorization (
    UserAuthorizationKey,
    ClassTime,
    IndividualProject,
    GroupMemberLastName,
    GroupMemberFirstName,
    GroupName
)
VALUES
(1, '10:45', 'create store procedures', 'Bains', 'Jasky', 'Group#1'),
(2, '10:45', 'drop foreign keys', 'Vathada', 'Mai', 'Group#1'),
(3, '10:45', 'load data', 'Garnica', 'Richard', 'Group#1'),
(4, '10:45', 'truncate tables', 'Wang', 'William', 'Group#1'),
(5, '10:45', 'Create first tables', 'Felix', 'Ashly', 'Group#1'),
(6, '10:45', 'load data', 'Cristobal', 'David', 'Group#1'),
(7, '10:45', 'default', 'default','default', 'default');


DO THE FOLLOWING FOR ALL TABLES

In [ ]:
ALTER TABLE [CH01-01-Dimension].DimCustomer
ADD UserAuthorizationKey INT NOT NULL DEFAULT 7;

ALTER TABLE [CH01-01-Dimension].DimGender
ADD UserAuthorizationKey INT NOT NULL DEFAULT 7;

ALTER TABLE [CH01-01-Dimension].DimMaritalStatus
ADD UserAuthorizationKey INT NOT NULL DEFAULT 7;

ALTER TABLE [CH01-01-Dimension].DimOccupation
ADD UserAuthorizationKey INT NOT NULL DEFAULT 7;

ALTER TABLE [CH01-01-Dimension].DimOrderDate
ADD UserAuthorizationKey INT NOT NULL DEFAULT 7;

ALTER TABLE [CH01-01-Dimension].DimProduct
ADD UserAuthorizationKey INT NOT NULL DEFAULT 7;

ALTER TABLE [CH01-01-Dimension].DimTerritory
ADD UserAuthorizationKey INT NOT NULL DEFAULT 7;

ALTER TABLE [CH01-01-Dimension].SalesManagers
ADD UserAuthorizationKey INT NOT NULL DEFAULT 7;

ALTER TABLE [CH01-01-Fact].Data
ADD UserAuthorizationKey INT NOT NULL DEFAULT 7;

ALTER TABLE [FileUpload].ProductCategories
ADD UserAuthorizationKey INT NOT NULL DEFAULT 7;

ALTER TABLE [FileUpload].ProductSubcategories
ADD UserAuthorizationKey INT NOT NULL DEFAULT 7;


YOU CAN CHECK that the tables now have a new column userauthorization by running the following:

In [ ]:
SELECT *
FROM [FileUpload].OriginallyLoadedData

create table process.workflows

In [ ]:
CREATE SCHEMA Process
CREATE TABLE Process.WorkflowSteps (
    WorkFlowStepKey INT NOT NULL PRIMARY KEY,
    WorkFlowStepDescription NVARCHAR(100) NOT NULL,
    WorkFlowStepTableRowCount INT NULL DEFAULT 0,
    StartingDateTime DATETIME2(7) NULL DEFAULT SYSDATETIME(),
    EndingDateTime DATETIME2(7) NULL DEFAULT SYSDATETIME(),
    ClassTime CHAR(5) NULL DEFAULT '10:45' CHECK (ClassTime = '10:45'),
    UserAuthorizationKey INT NOT NULL
);

now create a "function" that you can call everytimt you do smt you call this and decribe what you did and enter useridentificationkey

In [ ]:

CREATE OR ALTER PROCEDURE [Process].[usp_TrackWorkFlows]
    @StartTime DATETIME2,
    @WorkFlowDescription NVARCHAR(100),
    @WorkFlowStepTableRowCount INT,
    @UserAuthorizationKey INT
AS
BEGIN
    SET NOCOUNT ON;

    INSERT INTO Process.WorkflowSteps (
        WorkFlowStepKey,
        WorkFlowStepDescription,
        WorkFlowStepTableRowCount,
        StartingDateTime,
        EndingDateTime,
        ClassTime,
        UserAuthorizationKey
    )
    VALUES (
        (SELECT ISNULL(MAX(WorkFlowStepKey), 0) + 1 FROM Process.WorkflowSteps),
        @WorkFlowDescription,
        @WorkFlowStepTableRowCount,
        @StartTime,
        SYSDATETIME(),
        (SELECT ClassTime FROM DbSecurity.UserAuthorization WHERE UserAuthorizationKey = @UserAuthorizationKey),
        @UserAuthorizationKey
    );
END;


this is how we call the function

In [ ]:
DECLARE @StartTime DATETIME2 = SYSDATETIME();
DECLARE @RowCount INT;

SELECT @RowCount = COUNT(*) FROM [CH01-01-Dimension].DimProduct;

EXEC Process.usp_TrackWorkFlows
    @StartTime = @StartTime,
    @WorkFlowDescription = 'Created DbSecurity.UserAuthorization table', --describe what you did
    @WorkFlowStepTableRowCount = @RowCount,
    @UserAuthorizationKey = 5; -- use ur authorizationkey

<span style="color: #0000ff;">CREATE</span> <span style="color: #0000ff;">TABLE</span> \[CH01-01-Dimension\].DimProductCategory

In [ ]:
CREATE TABLE [CH01-01-Dimension].DimProductCategory (
    ProductCategoryKey INT NOT NULL PRIMARY KEY,
    CategoryName NVARCHAR(50) NOT NULL,
    UserAuthorizationKey INT NOT NULL,
    DateAdded DATETIME2 NULL DEFAULT SYSDATETIME(),
    DateOfLastUpdate DATETIME2 NULL DEFAULT SYSDATETIME(),
    CONSTRAINT FK_DimProductCategory_UserAuthorization 
        FOREIGN KEY (UserAuthorizationKey) 
        REFERENCES DbSecurity.UserAuthorization (UserAuthorizationKey)
);

<span style="color: #0000ff;">CREATE</span> <span style="color: #0000ff;">TABLE</span> \[CH01-01-Dimension\].DimProductSubcategory

In [ ]:
CREATE TABLE [CH01-01-Dimension].DimProductSubcategory (
    ProductSubcategoryKey INT NOT NULL PRIMARY KEY,
    SubcategoryName NVARCHAR(50) NOT NULL,
    ProductCategoryKey INT NOT NULL,
    UserAuthorizationKey INT NOT NULL,
    DateAdded DATETIME2 NULL DEFAULT SYSDATETIME(),
    DateOfLastUpdate DATETIME2 NULL DEFAULT SYSDATETIME(),
    CONSTRAINT FK_DimProductSubcategory_ProductCategory 
        FOREIGN KEY (ProductCategoryKey) 
        REFERENCES [CH01-01-Dimension].DimProductCategory (ProductCategoryKey),
    CONSTRAINT FK_DimProductSubcategory_UserAuthorization 
        FOREIGN KEY (UserAuthorizationKey) 
        REFERENCES DbSecurity.UserAuthorization (UserAuthorizationKey)
);

check foreign keys in db

In [ ]:
SELECT 
    fk.name AS ForeignKeyName,
    tp.name AS ParentTable,
    cp.name AS ParentColumn,
    tr.name AS ReferencedTable,
    cr.name AS ReferencedColumn
FROM sys.foreign_keys AS fk
JOIN sys.foreign_key_columns AS fkc 
    ON fk.object_id = fkc.constraint_object_id
JOIN sys.tables AS tp 
    ON fkc.parent_object_id = tp.object_id
JOIN sys.columns AS cp 
    ON fkc.parent_object_id = cp.object_id AND fkc.parent_column_id = cp.column_id
JOIN sys.tables AS tr 
    ON fkc.referenced_object_id = tr.object_id
JOIN sys.columns AS cr 
    ON fkc.referenced_object_id = cr.object_id AND fkc.referenced_column_id = cr.column_id
WHERE tp.name IN (
    'DimCustomer', 'DimGender', 'DimMaritalStatus', 'DimOccupation', 'DimOrderDate',
    'DimProduct', 'DimProductCategory', 'DimProductSubcategory', 'DimTerritory', 'SalesManagers',
    'Data', 'UserAuthorization', 'OriginallyLoadedData', 'ProductCategories', 
    'ProductSubcategories', 'WorkflowSteps'
);

drop all foreign keys

In [ ]:
ALTER TABLE [CH01-01-Fact].[Data] DROP CONSTRAINT FK_Data_SalesManagers;
ALTER TABLE [CH01-01-Fact].[Data] DROP CONSTRAINT FK_Data_DimGender;
ALTER TABLE [CH01-01-Dimension].[DimProductSubcategory] DROP CONSTRAINT FK_DimProductSubcategory_ProductCategory;
ALTER TABLE [CH01-01-Fact].[Data] DROP CONSTRAINT FK_Data_DimMaritalStatus;
ALTER TABLE [CH01-01-Fact].[Data] DROP CONSTRAINT FK_Data_DimOccupation;
ALTER TABLE [CH01-01-Fact].[Data] DROP CONSTRAINT FK_Data_DimOrderDate;
ALTER TABLE [CH01-01-Fact].[Data] DROP CONSTRAINT FK_Data_DimTerritory;
ALTER TABLE [CH01-01-Fact].[Data] DROP CONSTRAINT FK_Data_DimProduct;
ALTER TABLE [CH01-01-Fact].[Data] DROP CONSTRAINT FK_Data_DimCustomer;
ALTER TABLE [CH01-01-Dimension].[DimProductCategory] DROP CONSTRAINT FK_DimProductCategory_UserAuthorization;
ALTER TABLE [CH01-01-Dimension].[DimProductSubcategory] DROP CONSTRAINT FK_DimProductSubcategory_UserAuthorization;


truncate 13 tables

In [ ]:
TRUNCATE TABLE [CH01-01-Dimension].[DimCustomer];
TRUNCATE TABLE [CH01-01-Dimension].[DimGender];
TRUNCATE TABLE [CH01-01-Dimension].[DimMaritalStatus];
TRUNCATE TABLE [CH01-01-Dimension].[DimOccupation];
TRUNCATE TABLE [CH01-01-Dimension].[DimOrderDate];
TRUNCATE TABLE [CH01-01-Dimension].[DimProduct];
TRUNCATE TABLE [CH01-01-Dimension].[DimProductCategory];
TRUNCATE TABLE [CH01-01-Dimension].[DimProductSubcategory];
TRUNCATE TABLE [CH01-01-Dimension].[DimTerritory];
TRUNCATE TABLE [CH01-01-Dimension].[SalesManagers];
TRUNCATE TABLE [CH01-01-Fact].[Data];
TRUNCATE TABLE [FileUpload].[ProductCategories];
TRUNCATE TABLE [FileUpload].[ProductSubcategories];


check that we truncated tables

In [ ]:
SELECT 
    s.name AS SchemaName,
    t.name AS TableName,
    SUM(p.[rows]) AS [TotalRows]
FROM sys.tables t
JOIN sys.schemas s ON t.schema_id = s.schema_id
JOIN sys.partitions p ON t.object_id = p.object_id
WHERE p.index_id IN (0, 1)  -- 0 = heap, 1 = clustered index
GROUP BY s.name, t.name
ORDER BY s.name, t.name;


create the parsed table

In [ ]:
--DROP TABLE IF EXISTS ParsedFileUpload.OriginallyLoadedData;


CREATE SCHEMA ParsedFileUpload
-- Create the table
CREATE TABLE ParsedFileUpload.OriginallyLoadedData (
    SalesKey INT NOT NULL,
    ProductCategory VARCHAR(20) NULL,
    ProductSubcategory VARCHAR(20) NULL,
    SalesManager VARCHAR(20) NULL,
    ProductCode VARCHAR(10) NULL,
    ProductName VARCHAR(40) NULL,
    Color VARCHAR(10) NULL,
    ModelName VARCHAR(30) NULL,
    OrderQuantity INT NULL,
    UnitPrice MONEY NULL,
    ProductStandardCost MONEY NULL,
    SalesAmount MONEY NULL,
    OrderDate DATE NULL,
    MonthName VARCHAR(10) NULL,
    MonthNumber INT NULL,
    Year INT NULL,
    CustomerName VARCHAR(30) NULL,
    MaritalStatus CHAR(1) NULL,
    Gender CHAR(1) NULL,
    Education VARCHAR(20) NULL,
    Occupation VARCHAR(20) NULL,
    TerritoryRegion VARCHAR(20) NULL,
    TerritoryCountry VARCHAR(20) NULL,
    TerritoryGroup VARCHAR(20) NULL
);
GO

now load all data back

In [ ]:
INSERT INTO [CH01-01-Dimension].[DimCustomer] (
    CustomerName,
    UserAuthorizationKey
)
SELECT DISTINCT
    d.CustomerName, 5
      -- Replace with your actual UserAuthorizationKey
FROM FileUpload.OriginallyLoadedData d
WHERE d.CustomerName IS NOT NULL;


In [ ]:
INSERT INTO [CH01-01-Dimension].[DimGender] (
    Gender,
    GenderDescription,
    UserAuthorizationKey
)
SELECT DISTINCT
    d.Gender,
    CASE 
        WHEN d.Gender = 'M' THEN 'Male'
        WHEN d.Gender = 'F' THEN 'Female'
        ELSE 'Other'
    END,
    5
FROM FileUpload.OriginallyLoadedData d
WHERE d.Gender IS NOT NULL;


In [ ]:
INSERT INTO [CH01-01-Dimension].[DimMaritalStatus] (
    MaritalStatus,
    MaritalStatusDescription,
    UserAuthorizationKey
)
SELECT DISTINCT
    d.MaritalStatus,
    CASE 
        WHEN d.MaritalStatus = 'M' THEN 'Married'
        WHEN d.MaritalStatus = 'S' THEN 'Single'
        ELSE 'Other'
    END,
    5
FROM FileUpload.OriginallyLoadedData d
WHERE d.MaritalStatus IS NOT NULL;


In [ ]:
INSERT INTO [CH01-01-Dimension].[DimOccupation] (
    OccupationKey,
    Occupation,
    UserAuthorizationKey
)
SELECT 
    ROW_NUMBER() OVER (ORDER BY Occupation) AS OccupationKey,
    Occupation,
    5
FROM (
    SELECT DISTINCT d.Occupation
    FROM FileUpload.OriginallyLoadedData d
    WHERE d.Occupation IS NOT NULL
) AS UniqueOccupations;


In [ ]:
INSERT INTO [CH01-01-Dimension].[DimOrderDate] (
    OrderDate,
    MonthName,
    MonthNumber,
    Year,
    UserAuthorizationKey
)
SELECT DISTINCT
    d.OrderDate,
    d.MonthName,
    d.MonthNumber,
    d.Year,
    5
FROM FileUpload.OriginallyLoadedData d
WHERE d.OrderDate IS NOT NULL;
